# Modulus
This time we'll find the modulus by using a general addition and subtraction circuit.

The circuit is:

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Ff161e722-f727-b6fb-d02b-24e40b5b63b1.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=da298eace648814b5ccfe4affa9301ca">

$a,b$ is $0 < a,b < N$. The last bit is called the temporary bit. It checks the overflow.

## Implementation procedure
In order to find $a+b\mod N$, we need to compare $a+b$ and $N$.  

When $a+b>N$,  
$a+b-N = (a+b) \mod N$

When $a+b<N$, 
$a+b = (a+b) \mod N$

We will implement it in a quantum circuit.

When $a+b>N$,

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F5c97b6fd-425b-c42b-057a-68d9cb47e457.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=3711f239a543dc4aa175f4636adde888">

When $a+b<N$,

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Fcb7bc1bf-e713-f7d4-9cdc-4bc47bb6eba8.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=425f2f86ef9d5b5c27b1c1f1ddfab936">

It uses the surplus qubits and the value of the topmost qubit of the adder to make the case.  
If $a+b<N$, no extra bits are used and no extra operations are required.

## Example
Let's look at a simple example.

When $3 + 5 < 7$,  
$(3 + 5) \mod 7 = 1$

When $3 + 5 > 11$,    
$(3 + 5) \mod 11 = 8$

This is what we are trying to implement by using quantum circuits.

## Install Blueqat
Install Blueqat from pip.

In [1]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Implementation
We'll prepare it for implementation. This is the same as the addition circuit and the subtraction circuit.

we will add another N to the circuit above to make the temporary bit and N correspond to each other.   
The initial state is as follows.

```
c0 --
a0 --
b0 --
c1 --
..
n0 --
n1 --
..
t  --
n0 --
n1 --
..
```

Also, each digit is $N$.

In [118]:
from blueqat import Circuit

def carry(a):
    return Circuit().ccx[a+1,a+2,a+3].cx[a+1,a+2].ccx[a,a+2,a+3]

def carry_reverse(a):
    return Circuit().ccx[a,a+2,a+3].cx[a+1,a+2].ccx[a+1,a+2,a+3]

def sum(a):
    return Circuit().cx[a+1,a+2].cx[a,a+2] 

def sum_reverse(a):
    return Circuit().cx[a,a+2].cx[a+1,a+2]

def tobinary(A):
    return bin(A)[2:] 

# get the digit number
def digits2(a,b,n): 
     aa = tobinary(a)  
     bb = tobinary(b)  
     nn = tobinary(n)  

     nlen = len(nn)  
     aa = aa.zfill(nlen) 
     bb = bb.zfill(nlen) 

     str = '' 
     for i in range(nlen): 
         str += '0' + aa[nlen-i-1] + bb[nlen-i-1] 
     str += '0' 

     for i in range(nlen): 
         str += nn[nlen-i-1]  

     str += '0'

     for i in range(nlen): 
        str += nn[nlen-i-1]

     return str

# initial states
def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

# adder
def plus(a,b,n): 
     qubits = len(digits2(a,b,n))
     digi = len(tobinary(n))

     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry(i*3) 

     cir3 = Circuit(qubits).cx[(digi-1)*3+1,(digi-1)*3+2] + sum((digi-1)*3)

     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += carry_reverse((digi-i-2)*3) 
         cir4 += sum((digi-i-2)*3)

     cir_plus = cir2 + cir3 + cir4
     return cir_plus

# adder_inverse
def minus(a,ab,n): 
     qubits = len(digits2(a,ab,n))
     digi = len(tobinary(n))

     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += sum_reverse(i*3)
         cir4 += carry(i*3) 

     cir3 = sum_reverse((digi-1)*3) + Circuit(qubits).cx[(digi-1)*3+1,(digi-1)*3+2] 

     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry_reverse((digi-1-i)*3) 

     cir_minus = cir4 + cir3 + cir2
     return cir_minus

# swap a and N
def swap(n):
    digi = len(tobinary(n))
    cir = Circuit(5*digi+2)
    for i in range(digi):
        cir += Circuit(5*digi+2).swap[3*i+1,3*digi+1+i]
    return cir

# to decimal number
def todecimal(A):
    return int(str(A),2) 

# get b from the circuit
def getb(result,n): 
     str = ""
     digi = len(tobinary(n)) 
     for i in range(digi): 
         str += result[3*(digi-i)-1]
     return todecimal(str)

### General circuit

In [119]:
def adder_mod(a,b,n):
    digi = len(tobinary(n))

    # first part
    part1 = toX(digits2(a,b,n)) + plus(a,b,n) + swap(n) + minus(a,b,n)

    # set the overflow to a temporary bit
    part2 = Circuit(5*digi+2).x[digi*3].cx[digi*3,digi*4+1].x[digi*3]

    # returns N from a temporary bit.
    part3 = Circuit(5*digi+2)
    for i in range(digi):
        part3 += Circuit(5*digi+2).ccx[4*digi+2+i,4*digi+1,3*i+1]

    # last part
    part4 = minus(a,b,n)+Circuit(5*digi+2).cx[digi*3,digi*4+1]+plus(a,b,n)
    
    result = (part1+part2+part3+plus(a,b,n)+part3+swap(n)+part4).m[:].run(shots=1)
    return getb(result.most_common()[0][0],n)

Let's actually calculate it.

In [120]:
adder_mod(4,3,5)

2

In [121]:
adder_mod(4,4,5)

3

In [122]:
adder_mod(1,5,6)

0

We've calculated it.

### References
V. Vedral, A. Barenco, A. Ekert, "Quantum Networks for Elementary Arithmetic Operations",  
(Submitted on 16 Nov 1995)  
https://arxiv.org/pdf/quant-ph/9511018.pdf